In [258]:
%cd

/Users/jakbuck


/Users/jakbuck/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/Users/jakbuck/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:428: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [259]:
!python3 -m pip install pandas
!python3 -m pip install requests
!python3 -m pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [260]:
import dotenv
dotenv.load_dotenv()

False

In [261]:
import os
import requests

In [262]:

import pandas as pd
import re

In [263]:
df = pd.read_csv(os.path.expanduser("~/Documents/GitHub/q1-cs/foodstuffs/grade.csv"))

In [264]:
df

,Item Name,What kind of item is it?,Total number of ingredients,Name of first (main) ingredient,Serving Size,Calories per serving,Total Sodium,Total Carbohydrates,Total Fat,Total Protein,What allergens does it contain?,Does it contain GMO crops?,Where was it made?,How Processed is it?\n(see instructions for ranking)
0,Lemonade,Drink,4,Filtered Water,9 fl oz,110,20 mg,29 g,0g,0g,NaN,No,"Apoka, FL",2
1,Red Machine Naked Juice,Fridge,14,Apple Juice From Concentrate,15.2 Fl Oz,270 Cal,25 mg,56 g,4 g,4 g,NaN,No,U.S.A.,2
2,Gatorade(Cool Blue),Drink,12,Water,591 ml,140 Cal,270 mg,36 g,0 g,0 g,none,Yes,U.S.A.,4
3,Häagen-Dazs,Freezer,5,Cream,90g,210 Cal,45 mg,20 g,8 g,3 g,Skim milk and cane sugar,No,"Tilloy-lès-Mofflaines, France",3
4,Sour Cream,Fridge,1,Cultured Cream,30 g,60 Cal,15 mg,1 g,5 g,1 g,milk,Yes,U.S.A.,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,dried fruit,Snack,3,dried pears,40 g,140 cal,0 mg,33 g,0 g,1 g,none,no,"usa, peru",1.5
409,NaN,Snack,10,Unbleached Enriched Flour,16g,80,130mg,10g,4.5g,<1g,"Wheat, Soy.",Yes,Mexico,4
410,Trü Frü Frozen Chocolate Strawberries,Freezer,3,Strawberries,28g,90 Cal,15mg,10g,5g,1g,"milk, soy, tree nuts, peanuts",No,U.S.A.,2
411,Gatorade fruit punch + berry,Drink,13,Water,360mL,80,160mg,22g,0g,0g,NaN,Likely,Unknown but distributed in us,5


In [265]:
def lower_data(row):
    global df
    def try_lower(x):
        try:
            return x.lower()
        except AttributeError:
            return x
    df[row] = df[row].apply(try_lower)
def if_then(row,cond,func):
    global df
    def try_func(x):
        try:
            if cond(x):
                return func(x)
            return x
        except:
            return x
    df[row] = df[row].apply(try_func)

def remove_stuff(row,s):
    global df
    def try_lower(x):
        try:
            return x.replace(s,"")
        except AttributeError:
            return x
    df[row] = df[row].apply(try_lower)
def cleanify(unit, row, strict=True,put_none_if_no_match=True):
    global df
    if strict:
        df = df[df[row].str.contains(unit)]
    def clean(s):
        #https://regexr.com/85git amazing
        if strict:
            r = re.compile(f"([0-9]+\ ?(\.|\/)?\ ?[0-9]*)\ ?{unit}")
        else:
            r = re.compile(f"([0-9]+\ ?(\.|\/)?\ ?[0-9]*)\ ?({unit})?")
        matches = re.match(r, s)
        if matches:
            return matches.group(1)
        if put_none_if_no_match:
            return None
        return s

    
    df[row] = df[row].apply(clean)

def to_unit(thing,unit):
    request_string = f"{thing} to {unit}"
    resp = requests.get(f"https://api.wolframalpha.com/v1/result?i={request_string}&appid={os.getenv('WOLFRAM_API')}").text.split(" ")[1:]
    resp = "".join(resp).split("rams")[0]
    #print(resp)
    return resp

def clean_servingsize():
    global df
    df["Serving Size"] = (df["Serving Size"] + df["Item Name"]).apply(lambda x: to_unit(x,"grams"))

In [266]:
df = df.fillna("none")

In [267]:
#test

In [268]:
#clean_servingsize()
cleanify("g","Serving Size", False)

In [269]:
cleanify("cal","Calories per serving", False)

In [270]:
cleanify("mg","Total Sodium", False)

In [271]:
cleanify("g","Total Carbohydrates", False)

In [272]:
cleanify("g","Total Fat", False)

In [273]:
cleanify("g","Total Protein", False)

In [274]:
lower_data("Item Name")
lower_data("Name of first (main) ingredient")
lower_data("What allergens does it contain?")
lower_data("Does it contain GMO crops?")
lower_data("Where was it made?")

In [275]:
remove_stuff("Where was it made?", ".") # for stuff like u.s.a


In [276]:
df = df.replace({"Does it contain GMO crops?": {
    "no":0,
    "yes":1,
}})

df = df.replace({"What kind of item is it?": {
    "Drink":1,
    "Fridge":2,
    "Snack":3,
    "Canned/Dried":4,
    "Freezer":2
}})


/var/folders/tw/cm45tsj50pn501zrkvnzcywm0000gp/T/ipykernel_64446/3628664868.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"What kind of item is it?": {


In [277]:
def intfill(row):
    global df
    df[row] = pd.to_numeric(df[row].astype(str).str.replace(',',''), errors='coerce').fillna("none")#.astype(int)

In [278]:
intfill("Does it contain GMO crops?")
intfill("How Processed is it?\n(see instructions for ranking)")
intfill("Total number of ingredients")

In [279]:
if_then("What allergens does it contain?", lambda x: "n/a" in x, lambda x: "none")

In [280]:
df

,Item Name,What kind of item is it?,Total number of ingredients,Name of first (main) ingredient,Serving Size,Calories per serving,Total Sodium,Total Carbohydrates,Total Fat,Total Protein,What allergens does it contain?,Does it contain GMO crops?,Where was it made?,How Processed is it?\n(see instructions for ranking)
0,lemonade,1,4.0,filtered water,9,110,20,29,0,0,none,0.0,"apoka, fl",2.0
1,red machine naked juice,2,14.0,apple juice from concentrate,15.2,270,25,56,4,4,none,0.0,usa,2.0
2,gatorade(cool blue),1,12.0,water,591,140,270,36,None,0,none,1.0,usa,4.0
3,häagen-dazs,2,5.0,cream,90,210,45,20,8,3,skim milk and cane sugar,0.0,"tilloy-lès-mofflaines, france",3.0
4,sour cream,2,1.0,cultured cream,30,60,15,1,5,1,milk,1.0,usa,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,dried fruit,3,3.0,dried pears,40,None,0,33,0,1,none,0.0,"usa, peru",1.5
409,none,3,10.0,unbleached enriched flour,16,80,130,10,4.5,None,"wheat, soy.",1.0,mexico,4.0
410,trü frü frozen chocolate strawberries,2,3.0,strawberries,28,90,15,10,5,1,"milk, soy, tree nuts, peanuts",0.0,usa,2.0
411,gatorade fruit punch + berry,1,13.0,water,360,80,160,22,0,0,none,none,unknown but distributed in us,5.0


In [281]:
df.to_csv(os.path.expanduser("~/Documents/GitHub/q1-cs/foodstuffs/grade_cleaned.csv"))